### Функция получения данных

In [1]:
import warnings
import pandas as pd
import numpy as np
import json
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import empyrical as ep
import yfinance as yf
import statsmodels.api as sm
from sklearn import preprocessing
from scipy import stats
from scipy.signal import argrelextrema
import pandas_gbq
import sys, os
from google.cloud import bigquery
from google.oauth2 import service_account
from multiprocessing.pool import ThreadPool
import datetime
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import row
from bokeh.io import output_file, output_notebook
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Slope
from bokeh.models import PolyAnnotation
from bokeh.models.widgets import Tabs, Panel
import requests
import asyncio
import time
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def prepare_data_frame(bd_data, currency, sql_index):
    tickers = bd_data.index.tolist()
    #     print(tickers)
    Data_for_Portfolio_TOTAL = pd.DataFrame()

    for ticker in tickers:
        try:
            # =====================================================================
            io = json.loads(bd_data.loc[ticker].loc['annuals'].replace("'", '"'))
            pd.DataFrame(io['income_statement']).replace('No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('',
                                                                                                                     0).replace(
                'N/A', 0).astype(float)
            pre_date_list = pd.Series(io['Fiscal Year'])
            pre_date_list = pre_date_list.str.split("-")
            data = pre_date_list.to_list()
            names = ["Date", "Month"]
            atata = pd.DataFrame(data[:-1], columns=names)
            date_list = atata['Date'].append(pd.Series(data[-1]), ignore_index=True)
            income_df = pd.DataFrame(io['income_statement']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            balance_df = pd.DataFrame(io['balance_sheet']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            cashflow_df = pd.DataFrame(io['cashflow_statement']).set_index(date_list).replace('No Debt', 0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            valuation_ratios_df = pd.DataFrame(io['valuation_ratios']).set_index(date_list).replace('No Debt',
                                                                                                    0).replace \
                ('At Loss', 0).replace('-', 0).replace('', 0).replace('N/A', 0).astype(float)
            valuation_and_quality_df = pd.DataFrame(io['valuation_and_quality']).set_index(date_list).drop(
                ['Restated Filing Date', 'Filing Date', 'Earnings Release Date'], axis=1).replace('', 0).replace(
                'No Debt', 0).replace('At Loss', 0).replace('-', 0).replace('N/A', 0).astype(float)
            common_size_ratios_df = pd.DataFrame(io['common_size_ratios']).set_index(date_list).replace('No Debt', 0) \
                .replace('At Loss', 0).replace('', 0).replace('-', 0).replace('N/A', 0).replace(
                'Negative Tangible Equity',
                0).astype(float)
            per_share_data_df = pd.DataFrame(io['per_share_data_array']).set_index(date_list).replace('', 0) \
                .replace('No Debt', 0).replace('-', 0).replace('N/A', 0).astype(float)
            check = 1
        except:
            print('aaaaaaaaaaaaaa')
            check = 0
            pass
        
        if check == 1:
            try:
                Data_for_Portfolio = pd.DataFrame()

                Data_for_Portfolio['PE'] = valuation_ratios_df["PE Ratio"]
                Data_for_Portfolio['PB'] = valuation_ratios_df["PB Ratio"]
                Data_for_Portfolio['PS'] = valuation_ratios_df["PS Ratio"]
                Data_for_Portfolio['PEG'] = valuation_ratios_df["PEG Ratio"]
                Data_for_Portfolio['E/P'] = income_df['Net Income'] / valuation_and_quality_df['Market Cap']
                Data_for_Portfolio['Revenue'] = income_df['Revenue']
                Data_for_Portfolio['Net_Income'] = income_df['Net Income']
                Data_for_Portfolio['FCF_Margin_%'] = common_size_ratios_df['FCF Margin %']
                Data_for_Portfolio['Market Cap'] = valuation_and_quality_df['Market Cap']
                try:
                    Data_for_Portfolio['Shares Outstanding'] = valuation_and_quality_df['Shares Outstanding (EOP)']
                except:
                    pass
                Data_for_Portfolio['Intrinsic Value: Projected FCF'] = valuation_and_quality_df['Intrinsic Value: Projected FCF']
                Data_for_Portfolio['Net Margin %'] = income_df['Net Margin %']
                try:
                    Data_for_Portfolio['Operating Margin %'] = common_size_ratios_df['Operating Margin %']
                except:
                    pass
                Data_for_Portfolio['ROE %'] = common_size_ratios_df['ROE %']
                Data_for_Portfolio['ROA %'] = common_size_ratios_df['ROA %']
                Data_for_Portfolio['ROIC %'] = common_size_ratios_df['ROIC %']         
                Data_for_Portfolio['Dividend Yield %'] = valuation_ratios_df['Dividend Yield %']
                Data_for_Portfolio['YoY Rev. per Sh. Growth'] = valuation_and_quality_df['YoY Rev. per Sh. Growth']
                Data_for_Portfolio['YoY EPS Growth'] = valuation_and_quality_df['YoY EPS Growth']
                Data_for_Portfolio['Dividend Payout Ratio'] = common_size_ratios_df['Dividend Payout Ratio']   
                Data_for_Portfolio['Debt-to-Equity'] = balance_df['Debt-to-Equity']  
                
                try:
                    Data_for_Portfolio['market_cap'] = (valuation_and_quality_df['Shares Outstanding (EOP)'][-1] * 1000000) * \
                             yahoo_df[ticker].dropna()[-1]
                except:
                    pass
                Data_for_Portfolio['Company'] = ticker
                Data_for_Portfolio['Date'] = date_list.tolist()              

                Data_for_Portfolio = Data_for_Portfolio.replace([np.inf, -np.inf], 0)
                Data_for_Portfolio = Data_for_Portfolio.set_index('Company')
                Data_for_Portfolio = Data_for_Portfolio[::-1].fillna(0)

                Data_for_Portfolio_TOTAL = pd.concat([Data_for_Portfolio, Data_for_Portfolio_TOTAL])

                sys.stdout.write("\r" + str(ticker))
                sys.stdout.flush()
                time.sleep(0.3)
                
            except Exception as e:
                print(e)
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(exc_type, fname, exc_tb.tb_lineno)
                pass

    return Data_for_Portfolio_TOTAL

def yahoo_adj_close(exchange, project_id):
    print('yahoo_adj_close')
    sql_adj_close = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Adj_Close_2`
    WHERE _Exchange in {exchange}
    """
    sql_yahoo_data_adj_close = pandas_gbq.read_gbq(sql_adj_close, project_id=project_id)
    pre_adj_close = sql_yahoo_data_adj_close.rename(columns=lambda x: x.replace('_', '-')[1:])
    pre_adj_close = pre_adj_close.T.rename(columns=sql_yahoo_data_adj_close['_Ticker'])[:-3]
    # Converting the index as date
    pre_adj_close.index = pd.to_datetime(pre_adj_close.index, format='%Y-%m-%d')
    sql_yahoo_data_adj_close = pre_adj_close[:-1]
    return sql_yahoo_data_adj_close

def yahoo_adj_close_list(company, project_id):
    print('yahoo_adj_close')
    sql_adj_close = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Adj_Close_2`
    WHERE _Ticker in {company}
    """
    sql_yahoo_data_adj_close = pandas_gbq.read_gbq(sql_adj_close, project_id=project_id)
    pre_adj_close = sql_yahoo_data_adj_close.rename(columns=lambda x: x.replace('_', '-')[1:])
    pre_adj_close = pre_adj_close.T.rename(columns=sql_yahoo_data_adj_close['_Ticker'])[:-3]
    # Converting the index as date
    pre_adj_close.index = pd.to_datetime(pre_adj_close.index, format='%Y-%m-%d')
    sql_yahoo_data_adj_close = pre_adj_close[:-1]
    return sql_yahoo_data_adj_close

def sql_index(project_id):
    sql_index_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.INDEXES`
    """
    sql_index = pandas_gbq.read_gbq(sql_index_req, project_id=project_id)
    return sql_index

def sql_guru(exchange, project_id):
    if len(exchange) >= 2:
        sql_guru_req = f"""
        SELECT *
        FROM `buoyant-apogee-281013.FIN.GURU`
        WHERE exchange in {exchange}
        """
    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    return sql_data_guru

def sql_guru_list(company, project_id):
    sql_guru_req = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.GURU`
    WHERE company in {company}
    """
    sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
    return sql_data_guru

def sql_guru_yahoo_table(project_id):
    sql_open = f"""
    SELECT *
    FROM `buoyant-apogee-281013.FIN.Yahoo_Exchange`
    """
    bd_data_yahoo = pandas_gbq.read_gbq(sql_open, project_id=project_id)
    return bd_data_yahoo

def sql_yahoo(ticker_list, guru_yahoo_table):
    tickers = []
    tickers_name_final = []
    for tik in ticker_list:
        #print(tik)
        if tik.split(':')[0] == 'NAS' or tik.split(':')[0] == 'NYSE':
            yah_exc = ''
        elif tik.split(':')[0] == 'AMEX':
            yah_exc = ''
        elif tik.split(':')[0] == 'HKSE':
            yah_exc = guru_yahoo_table['yahoo'].where(guru_yahoo_table['guru'] == tik.split(':')[0]).dropna().values[0]
        else:
            yah_exc = guru_yahoo_table['yahoo'].where(guru_yahoo_table['guru'] == tik.split(':')[0]).dropna().values[0]

        if tik.split(':')[0] == 'HKSE':
            tickers.append(tik.split(':')[1][1:] + yah_exc)
        else:
            tickers.append(tik.split(':')[1] + yah_exc)

    #print(tickers)
    try:
        yahoo_data = yf.download(tickers)
        # yahoo_data = yahoo_data[:limit_date]
        yahoo_data_check = yahoo_data['Adj Close']
    except:
        pass
    tickers_name_yahoo = []

    for tik_yahoo in yahoo_data_check.columns.tolist():

        if tik_yahoo.split('.')[-1] == 'HK':
            for i in range(len(ticker_list)):
                if tik_yahoo.split('.')[0] == ticker_list[i].split(':')[1][1:].split('.')[0]:
                    tickers_name_final.append(ticker_list[i])
                    tickers_name_yahoo.append(tik_yahoo)

        else:
            for i in range(len(ticker_list)):
                if tik_yahoo.split('.')[0] == ticker_list[i].split(':')[1].split('.')[0]:
                    tickers_name_final.append(ticker_list[i])
                    tickers_name_yahoo.append(tik_yahoo)

    #print('tickers_name_yahoo')
    #print(tickers_name_yahoo)
    yahoo_adj_close = pd.DataFrame()
    yahoo_adj_close = yahoo_data['Adj Close'][tickers_name_yahoo].copy().set_axis(tickers_name_final, axis=1)
    yahoo_close = yahoo_data['Close'][tickers_name_yahoo].copy().set_axis(tickers_name_final, axis=1)


    #print('yahoo_adj_close')
    #print(yahoo_adj_close)

    return yahoo_adj_close, yahoo_close

In [ ]:
type_of_get_data = 'From_List'
cheked_year = '2007'
cheked_year_end = '2021'
currency = 'EUR=X'

# *****************************SQL params ***************
credentials = service_account.Credentials.from_service_account_file('gadgets_database_key.json')
project_id = 'buoyant-apogee-281013'
client = bigquery.Client(credentials=credentials, project=project_id)
table_id_guru = "buoyant-apogee-281013.FIN.GURU"
table_id_yahoo_adj_close = "buoyant-apogee-281013.FIN.Adj Close"
job_config = bigquery.QueryJobConfig()

companies = pd.read_excel('Tinkoff.xlsx', sheet_name='US')['США (SP500 + насдак 100)']
companies = tuple(companies)
print(companies[:10], '...')
print(len(companies))

# ===================== treads for qsl requests
pool = ThreadPool(processes=8)

return_sql_data_guru = pool.apply_async(sql_guru_list, (companies, project_id))
return_sql_yahoo_data_adj_close = pool.apply_async(yahoo_adj_close_list, (companies, project_id))
return_sql_index = pool.apply_async(sql_index, ([project_id]))
# do some other stuff in the main process
sql_yahoo_data_adj_close = return_sql_yahoo_data_adj_close.get()
#sql_index = return_sql_index.get()
sql_data_guru = return_sql_data_guru.get()

# *****************************SQL requests ***************
#index = sql_index['index'][0]
tickers_yahoo = sql_yahoo_data_adj_close.columns.tolist()
tickers_guru = sql_yahoo_data_adj_close.columns.tolist()

data_guru = sql_data_guru[sql_data_guru['company'].isin(tickers_guru)].set_index('company')
guru_yahoo_table = sql_guru_yahoo_table(project_id)

yahoo_adj_close= sql_yahoo(companies, guru_yahoo_table)
yahoo_df = pd.DataFrame(yahoo_adj_close[0])
#------------------------------------------------------------------------------------------------
Data_for_Portfolio_TOTAL = pd.DataFrame()
Data_for_Portfolio_TOTAL = prepare_data_frame(data_guru, currency, sql_index)

print(' DONE!')
tickers = data_guru.index.tolist()

('NYSE:A', 'NAS:AAL', 'NYSE:AAP', 'NAS:AAPL', 'NYSE:ABBV', 'NYSE:ABC', 'NAS:ABMD', 'NYSE:ABT', 'NYSE:ACN', 'NAS:ADBE') ...
524
yahoo_adj_close




Downloading:   0%|          | 0/23 [00:00<?, ?rows/s]

Downloading: 100%|██████████| 23/23 [00:00<00:00, 74.13rows/s]


Downloading:   0%|          | 0/524 [00:00<?, ?rows/s]


Downloading:  32%|███▏      | 167/524 [00:46<01:41,  3.52rows/s]

Downloading:  40%|███▉      | 209/524 [00:59<01:30,  3.49rows/s][A

Downloading:  48%|████▊     | 249/524 [01:10<01:19,  3.45rows/s][A


Downloading:  66%|██████▌   | 345/524 [01:13<00:36,  4.87rows/s]

Downloading:  55%|█████▌    | 289/524 [01:32<01:25,  2.75rows/s]

Downloading:  63%|██████▎   | 332/524 [01:46<01:08,  2.82rows/s]

Downloading:  40%|███▉      | 209/524 [01:06<01:37,  3.23rows/s]


Downloading:  95%|█████████▌| 500/524 [01:50<00:05,  4.65rows/s]

Downloading:  78%|███████▊  | 409/524 [02:12<00:39,  2.88rows/s]

Downloading:  86%|████████▋ | 452/524 [02:27<00:25,  2.86rows/s]


Downloading:  95%|█████████▌| 500/524 [01:45<00:05,  4.68rows/s]

Downloading:  63%|██████▎   | 332/524 [01:48<01:03,  3.03rows/s]


Downloading:  93%|███

[*********************100%***********************]  524 of 524 completed
NYSE:CTLT DONE!


In [ ]:
#Data_for_Portfolio_TOTAL.describe()

In [ ]:
list_of_control = ['PE', 'PB', 'PS', 'PEG', 'FCF_Margin_%', 'Net Margin %', 'Operating Margin %', 'ROE %', 'ROA %', 'ROIC %', 'Dividend Yield %', 'Dividend Payout Ratio', \
                  'YoY Rev. per Sh. Growth', 'YoY EPS Growth']
for column in list_of_control:
    for i in range(len(Data_for_Portfolio_TOTAL)):
        if Data_for_Portfolio_TOTAL[column][i] > abs(Data_for_Portfolio_TOTAL[column].mean())*12:
            Data_for_Portfolio_TOTAL[column][i] = 0
        elif Data_for_Portfolio_TOTAL[column][i] < 0:
            Data_for_Portfolio_TOTAL[column][i] = 0

In [ ]:
#Data_for_Portfolio_TOTAL.describe()

In [ ]:
Data_for_Portfolio_TOTAL['FCF'] = Data_for_Portfolio_TOTAL['Revenue']*Data_for_Portfolio_TOTAL['FCF_Margin_%']
Data_for_Portfolio_TOTAL['Dividends payout'] = Data_for_Portfolio_TOTAL['Net_Income']*Data_for_Portfolio_TOTAL['Dividend Payout Ratio']
Data_for_Portfolio_TOTAL = Data_for_Portfolio_TOTAL[Data_for_Portfolio_TOTAL['Date']!='2021']
Data_for_Portfolio_TOTAL['Date'] = Data_for_Portfolio_TOTAL['Date'].replace('TTM', '2021')

In [ ]:
portfolio_df = pd.DataFrame()
portfolio_df['PE'] = Data_for_Portfolio_TOTAL.groupby('Date')['PE'].mean()
portfolio_df['PB'] = Data_for_Portfolio_TOTAL.groupby('Date')['PB'].mean()
portfolio_df['PS'] = Data_for_Portfolio_TOTAL.groupby('Date')['PS'].mean()
portfolio_df['PEG'] = Data_for_Portfolio_TOTAL.groupby('Date')['PEG'].mean()

In [ ]:
portfolio_df['Revenue'] = Data_for_Portfolio_TOTAL.groupby('Date')['Revenue'].mean()
portfolio_df['Net_Income'] = Data_for_Portfolio_TOTAL.groupby('Date')['Net_Income'].mean()
portfolio_df['FCF'] = Data_for_Portfolio_TOTAL.groupby('Date')['FCF'].mean()
portfolio_df['Market Cap'] = Data_for_Portfolio_TOTAL.groupby('Date')['Market Cap'].sum()

In [ ]:
cagr_period = 5
portfolio_df['Revenue CAGR'] = ''
portfolio_df['Net_Income CAGR'] = ''
for i in range(4,len(portfolio_df)):
    portfolio_df['Revenue CAGR'][i] = (portfolio_df['Revenue'][i]/portfolio_df['Revenue'][i-cagr_period]) ** (1/cagr_period) - 1
    portfolio_df['Net_Income CAGR'][i] = (portfolio_df['Net_Income'][i]/portfolio_df['Net_Income'][i-cagr_period]) ** (1/cagr_period) - 1

In [ ]:
portfolio_df['Net Margin %'] = Data_for_Portfolio_TOTAL.groupby('Date')['Net Margin %'].mean()
portfolio_df['Operating Margin %'] = Data_for_Portfolio_TOTAL.groupby('Date')['Operating Margin %'].mean()
portfolio_df['FCF_Margin_%'] = Data_for_Portfolio_TOTAL.groupby('Date')['FCF_Margin_%'].mean()
portfolio_df['ROE %'] = Data_for_Portfolio_TOTAL.groupby('Date')['ROE %'].mean()
portfolio_df['ROA %'] = Data_for_Portfolio_TOTAL.groupby('Date')['ROA %'].mean()
portfolio_df['ROIC %'] = Data_for_Portfolio_TOTAL.groupby('Date')['ROIC %'].mean()
portfolio_df['Dividend Yield %'] = Data_for_Portfolio_TOTAL.groupby('Date')['Dividend Yield %'].mean()
portfolio_df['YoY Rev. per Sh. Growth'] = Data_for_Portfolio_TOTAL.groupby('Date')['YoY Rev. per Sh. Growth'].mean()
portfolio_df['YoY EPS Growth'] = Data_for_Portfolio_TOTAL.groupby('Date')['YoY EPS Growth'].mean()
portfolio_df['Dividend Yeld'] = Data_for_Portfolio_TOTAL.groupby('Date')['Dividends payout'].sum()/Data_for_Portfolio_TOTAL.groupby('Date')['Market Cap'].sum()
portfolio_df['Debt-to-Equity'] = Data_for_Portfolio_TOTAL.groupby('Date')['Debt-to-Equity'].mean()

In [ ]:
#portfolio_df = portfolio_df[(portfolio_df.index.month == 12)]

In [ ]:
portfolio_df['Revenue Growth'] = portfolio_df['Revenue'].pct_change()
portfolio_df['Net_Income Growth'] = portfolio_df['Net_Income'].pct_change()
portfolio_df['FCF Growth'] = portfolio_df['FCF'].pct_change()

In [ ]:
portfolio_df['SMA Market Cap'] = portfolio_df['Market Cap'].rolling(5).mean()
portfolio_df['SMA PE'] = portfolio_df['PE'].rolling(5).mean()
portfolio_df['SMA Net_Income Growth'] = portfolio_df['Net_Income Growth'].rolling(5).mean()

In [ ]:
portfolio_df = portfolio_df[(portfolio_df.index >= cheked_year) & (portfolio_df.index <= cheked_year_end)]
portfolio_df['Market Cap'][-1] =  Data_for_Portfolio_TOTAL_2.groupby('Date')['market_cap'].sum()[-1]/1000000

### Построение графиков 

In [21]:
output_notebook()
p = figure(title='График PE',
             plot_height=350, plot_width=650,
             x_axis_label='Год', y_axis_label='PE', x_axis_type="linear", toolbar_location='above')
p.line(portfolio_df.index, portfolio_df['PE'], color='green', line_width=1)
o = figure(title='График PB',
             plot_height=350, plot_width=650,
             x_axis_label='Год', y_axis_label='PB', x_axis_type="linear", toolbar_location='above')
o.line(portfolio_df.index, portfolio_df['PB'], color='green', line_width=1)
q = figure(title='График PS',
             plot_height=350, plot_width=650,
             x_axis_label='Год', y_axis_label='PS', x_axis_type="linear", toolbar_location='above')
q.line(portfolio_df.index, portfolio_df['PS'], color='green', line_width=1)
r = figure(title='График PEG',
             plot_height=350, plot_width=650,
             x_axis_label='Год', y_axis_label='PEG', x_axis_type="linear", toolbar_location='above')
r.line(portfolio_df.index, portfolio_df['PEG'], color='green', line_width=1)
p_panel = Panel(child=p, title='PE')
o_panel = Panel(child=o, title='PB')
q_panel = Panel(child=q, title='PS')
r_panel = Panel(child=r, title='PEG')
tabs = Tabs(tabs=[p_panel, o_panel, q_panel, r_panel])
show(tabs)
k = figure(title='График Доходности',
             plot_height=400, plot_width=500,
             x_axis_label='Год', y_axis_label='Доходность', x_axis_type="linear", toolbar_location='above')
par = np.polyfit(portfolio_df.index.astype(float), portfolio_df['Revenue'], 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in portfolio_df.index.astype(float)]
k.line(portfolio_df.index, portfolio_df['Revenue'], color='green', line_width=1, legend='Revenue')
k.line(portfolio_df.index, y_predicted, color='green', line_width=0.5, line_dash='dashed')
y_column2_range = 'CAGR'+ "_range"
k.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['Revenue CAGR'].min() * (2 - 0.1), end=portfolio_df['Revenue CAGR'].max() * (1 + 0.1))}
k.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
k.line(portfolio_df.index, portfolio_df['Revenue CAGR'], legend='Revenue CAGR', line_width=1, y_range_name=y_column2_range,  color="palegreen")
k.legend.location = 'top_left'
m = figure(title='График Прибыли',
             plot_height=400, plot_width=500,
             x_axis_label='Год', y_axis_label='Прибыль', x_axis_type="linear", toolbar_location='above')
par1 = np.polyfit(portfolio_df.index.astype(float), portfolio_df['Net_Income'], 1, full=True)
slope1=par1[0][0]
intercept1=par1[0][1]
y_predicted1 = [slope1*i + intercept1  for i in portfolio_df.index.astype(float)]
m.line(portfolio_df.index, portfolio_df['Net_Income'], color='red', line_width=1, legend='Net_Income')
m.line(portfolio_df.index, y_predicted1, color='red', line_width=0.5, line_dash='dashed')
y_column2_range = 'CAGR'+ "_range"
m.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['Net_Income CAGR'].min() * (1 - 0.1), end=portfolio_df['Net_Income CAGR'].max() * (1 + 0.1))}
m.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
m.line(portfolio_df.index, portfolio_df['Net_Income CAGR'], legend='Net_Income CAGR', line_width=1, y_range_name=y_column2_range,  color="lightpink")
m.legend.location = 'top_left'
n = figure(title='График Доходности цепной',
             plot_height=400, plot_width=500,
             x_axis_label='Год', y_axis_label='Доходность', x_axis_type="linear", toolbar_location='above')
n.line(portfolio_df.index, portfolio_df['Revenue']*100/portfolio_df['Revenue'][0], color='green', line_width=1, legend='Revenue')
n.line(portfolio_df.index, portfolio_df['Net_Income']*100/portfolio_df['Net_Income'][0], color='red', line_width=1, legend='Net_Income')
n.line(portfolio_df.index, portfolio_df['FCF']*100/portfolio_df['FCF'][0], color='blue', line_width=1, legend='FCF')
n.line(portfolio_df.index, portfolio_df['Market Cap']*100/portfolio_df['Market Cap'][0], color='orange', line_width=1, legend='Market Cap')
n.legend.location = 'top_left'
l = figure(title='График Роста',
             plot_height=400, plot_width=500,
             x_axis_label='Год', y_axis_label='Рост', x_axis_type="linear", toolbar_location='above')
l.line(portfolio_df.index, portfolio_df['YoY Rev. per Sh. Growth'], color='green', line_width=1, legend='YoY Rev. per Sh. Growth')
l.line(portfolio_df.index, portfolio_df['YoY EPS Growth'], color='red', line_width=1, legend='YoY EPS Growth')
l.legend.location = 'top_left'
show(row(k,m))
show(row(n,l))
s = figure(title='График Маржинальностей',
             plot_height=400, plot_width=650,
             x_axis_label='Год', y_axis_label='Margin %', x_axis_type="linear", toolbar_location='above')
s.line(portfolio_df.index, portfolio_df['Net Margin %'], color='green', line_width=1, legend='Net Margin %')
s.line(portfolio_df.index, portfolio_df['Operating Margin %'], color='blue', line_width=1, legend='Operating Margin %')
s.line(portfolio_df.index, portfolio_df['FCF_Margin_%'], color='red', line_width=1, legend='FCF_Margin_%')
s.legend.location = 'bottom_left'
t = figure(title='График Эффективностей',
             plot_height=400, plot_width=650,
             x_axis_label='Год', y_axis_label='%', x_axis_type="linear", toolbar_location='above')
t.line(portfolio_df.index, portfolio_df['ROE %'], color='green', line_width=1, legend='ROE %')
t.line(portfolio_df.index, portfolio_df['ROA %'], color='blue', line_width=1, legend='ROA %')
t.line(portfolio_df.index, portfolio_df['ROIC %'], color='red', line_width=1, legend='ROIC %')
t.legend.location = 'bottom_left'
u = figure(title='График объема Дивидендов',
             plot_height=400, plot_width=650,
             x_axis_label='Год', y_axis_label='%', toolbar_location='above')
u.line(portfolio_df.index, portfolio_df['Dividend Yeld'], color='green', line_width=1, legend='Dividend Yeld')
u.legend.location = 'bottom_left'
v = figure(title='График Debt-to-Equity',
             plot_height=400, plot_width=650,
             x_axis_label='Год', y_axis_label='%', toolbar_location='above')
v.line(portfolio_df.index, portfolio_df['Debt-to-Equity'], color='green', line_width=1, legend='Debt-to-Equity')
v.legend.location = 'bottom_left'
s_panel = Panel(child=s, title='Маржинальности')
t_panel = Panel(child=t, title='Эффективность')
u_panel = Panel(child=u, title='Дивиденды')
v_panel = Panel(child=v, title='Debt-to-Equity')
tabs2 = Tabs(tabs=[s_panel, t_panel, u_panel, v_panel])
show(tabs2)
w = figure(title='График Сравнения',
             plot_height=400, plot_width=900,
             x_axis_label='Год', x_axis_type="linear")
#w.line(portfolio_df.index, portfolio_df['SMA Market Cap'], color='orange', line_width=1, legend='SMA Market Cap')
w.line(portfolio_df.index, portfolio_df['Market Cap'], color='orange', line_width=1, legend='Market Cap')
#w.line(portfolio_df.index, portfolio_df['Net_Income Growth']*1000, legend='Net_Income', line_width=1, color="blue")
# SECOND AXIS
y_column2_range = 'PE'+ "_range"
w.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['PE'].min() * (1 - 0.1), end=portfolio_df['PE'].max() * (1 + 0.1))}
w.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
w.line(portfolio_df.index, portfolio_df['SMA PE'], legend='SMA PE', line_width=1, y_range_name=y_column2_range,  color="green")
w.line(portfolio_df.index, portfolio_df['PE'], legend='PE', line_width=1, y_range_name=y_column2_range, color="blue")
#w.line(portfolio_df.index, portfolio_df['SMA Net_Income Growth']/portfolio_df['SMA Net_Income Growth'][0], legend='SMA Net_Income Growth', line_width=1, color="blue")
w.legend.location = 'top_left'
n = figure(title='График Ростов',
             plot_height=300, plot_width=900,
             x_axis_label='Год', y_axis_label='Net_Income', x_axis_type="linear")
n.line(portfolio_df.index, portfolio_df['Net_Income'], color='red', line_width=1, legend='Net_Income')
# SECOND AXIS
# y_column2_range = 'SMA Net_Income Growth'+ "_range"
# n.extra_y_ranges = {y_column2_range: Range1d(start=portfolio_df['SMA Net_Income Growth'].min() * (1 - 0.1), end=portfolio_df['SMA Net_Income Growth'].max() * (1 + 0.1))}
# n.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
# n.line(portfolio_df.index, portfolio_df['SMA Net_Income Growth'], color='red', y_range_name=y_column2_range, line_width=1, legend='SMA Net_Income Growth')
# n.legend.location = 'top_left'
show(w)
show(n)

company = companies

sql_guru_req = f"""
SELECT *
FROM `buoyant-apogee-281013.FIN.Summary`
WHERE company in {company}
"""

sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
sql_data_guru = sql_data_guru.set_index('company')

tickers = sql_data_guru.index.tolist()
company_data_df = []

for ticker in tickers:
#     =====================================================================
    company_data = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['company_data']), orient='index').T
    company_data_df.append(company_data.iloc[0])
    # chart = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['chart']), orient='index').T              
    # general = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['general']), orient='index').T 
    # estimate = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['estimate']), orient='index').T 
    # guru = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['guru']), orient='index').T 
    # insider = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['insider']), orient='index').T 
    # ratio = pd.DataFrame.from_dict(eval(sql_data_guru.loc[ticker].loc['ratio']), orient='index').T 
company_data_df = pd.DataFrame(company_data_df)
company_data_df['rank_profitability'] = company_data_df['rank_profitability']/10
company_data_df['p2gf_value'] = company_data_df['p2gf_value'].astype('float')
print(company_data_df[['p2gf_value', 'rank_financial_strength', 'rank_profitability', 'rank_value']].mean().round(2))


# *****************************SQL params ***************
credentials = service_account.Credentials.from_service_account_file('gadgets_database_key.json')
project_id = 'buoyant-apogee-281013'
client = bigquery.Client(credentials=credentials, project=project_id)
table_id_guru = "buoyant-apogee-281013.FIN.GURU"
table_id_yahoo_adj_close = "buoyant-apogee-281013.FIN.Summary"
job_config = bigquery.QueryJobConfig()

exchange = ('10-Year Treasury Constant Maturity Rate', '1-Year Treasury Constant Maturity Rate', \
            'Securities held in Custody for Foreign Official / International Accounts: Marketable U.S. Treasury S', \
           'U.S. Treasury Securities held by Federal Reserve: All Maturities', 'University of Michigan: Inflation Expectation', \
           'Trade Balance: Goods and Services, Balance of Payments Basis', \
            'Federal Debt held by Federal Reserve Banks', 'Federal Debt held by Foreign and International Investors', \
           'Gross Domestic Product (GDP)', 'Total US population', 'Shiller PE Ratio for the S&P 500', 'Ratio of Wilshire 5000 over GNP', \
            'NYSE Investor Margin Debt Relative To GDP', \
            'Shiller PE Ratio Implied Market Return for the S&P 500', 'Tobin Q', 'Insider Buy/Sell Ratio - Overall Market', \
            'SP 500 Index', 'SP 500 GF Value', 'CBOE Volatility Index (the VIX)')

sql_guru_req = f"""
SELECT *
FROM `buoyant-apogee-281013.FIN.Economic_Indicators`
WHERE Indicator in {exchange}
"""

sql_data_guru = pandas_gbq.read_gbq(sql_guru_req, project_id=project_id)
sql_data_guru = sql_data_guru.set_index('Indicator')

# indicator_list = ['Global price of Fish', 'Gross Private Saving', 'Global price of Tin', 'Global price of Zinc']
sql_data_guru['description'] # обращение к описанию отрасли
final_df = pd.DataFrame()

for indicator in sql_data_guru.index.tolist():
    io = json.loads(sql_data_guru.loc[indicator].loc['data'].replace('"', '~').replace("None", '"0"').replace("'", '"').replace("True", '"True"').replace("False", '"False"'))
    pr_df = pd.DataFrame(io, columns=['year', indicator]).set_index('year')
    final_df = final_df.join(pr_df, how='outer')
final_df.index = pd.to_datetime(final_df.index)
final_df = final_df[(final_df.index >= cheked_year) & (final_df.index <= cheked_year_end)]
#final_df.tail()

treasury1 = final_df['10-Year Treasury Constant Maturity Rate'].dropna() - final_df['1-Year Treasury Constant Maturity Rate'].dropna()
x = figure(title='10-1 Year Treasury Constant Maturity Rate', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
x.line(treasury1.index, treasury1, color='green', line_width=1, legend='Treasury')
x.legend.location = 'top_left'
treasury2 = final_df['Securities held in Custody for Foreign Official / International Accounts: Marketable U.S. Treasury S'].dropna()/final_df['Securities held in Custody for Foreign Official / International Accounts: Marketable U.S. Treasury S'].dropna()[0]
treasury3 = final_df['U.S. Treasury Securities held by Federal Reserve: All Maturities'].dropna()/final_df['U.S. Treasury Securities held by Federal Reserve: All Maturities'].dropna()[0]
y = figure(title='Цепной индекс Treasury Securities', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
y.line(treasury2.index, treasury2, color='green', line_width=1, legend='Securities held in Custody for Foreign Official')
y.line(treasury3.index, treasury3, color='blue', line_width=1, legend='U.S. Treasury Securities')
y.legend.location = 'top_left'
show(row(x,y))
inflation = final_df['University of Michigan: Inflation Expectation'].dropna()
z = figure(title='University of Michigan: Inflation Expectation', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
z.line(inflation.index, inflation, color='green', line_width=1, legend='Inflation')
z.legend.location = 'top_left'
balans = final_df['Trade Balance: Goods and Services, Balance of Payments Basis'].dropna()
a = figure(title='Trade Balance: Goods and Services, Balance of Payments Basis', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
a.line(balans.index, balans, color='green', line_width=1, legend='Trade Balance')
a.legend.location = 'top_left'
show(row(z,a))
debt1 = (final_df['Federal Debt held by Federal Reserve Banks'].dropna()/final_df['Federal Debt held by Federal Reserve Banks'].dropna()[0])
debt2 = (final_df['Federal Debt held by Foreign and International Investors'].dropna()/final_df['Federal Debt held by Foreign and International Investors'].dropna()[0])
b = figure(title='Цепной индекс Federal Debt', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
b.line(debt1.index, debt1, color='green', line_width=1, legend='Federal Debt held by Federal Reserve Banks')
b.line(debt2.index, debt2, color='blue', line_width=1, legend='Federal Debt held by Foreign and International Investors')
b.legend.location = 'top_left'
gdp = final_df['Gross Domestic Product (GDP)'].dropna()
c = figure(title='Gross Domestic Product (GDP)', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
c.line(gdp.index, gdp, color='green', line_width=1, legend='Gross Domestic Product (GDP)')
c.legend.location = 'top_left'
show(row(b,c))
population = final_df['Total US population'].dropna()
d = figure(title='Total US population', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
d.line(population.index, population, color='green', line_width=1, legend='Total US population')
d.legend.location = 'top_left'
show(d)
pe_sp500 = final_df['Shiller PE Ratio for the S&P 500'].dropna()
pe_sp500_imlied_market = final_df['Shiller PE Ratio Implied Market Return for the S&P 500'].dropna()
e = figure(title='Shiller PE Ratio for the S&P 500', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
e.line(pe_sp500.index, pe_sp500, color='green', line_width=1, legend='Shiller PE Ratio for the S&P 500')
e.line(pe_sp500_imlied_market.index, pe_sp500_imlied_market, color='blue', line_width=1, legend='Shiller PE Ratio Implied Market Return for the S&P 500')
e.legend.location = 'top_left'
wilshire = final_df['Ratio of Wilshire 5000 over GNP'].dropna()
f = figure(title='Ratio of Wilshire 5000 over GNP', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
f.line(wilshire.index, wilshire, color='green', line_width=1, legend='Ratio of Wilshire 5000 over GNP')
f.legend.location = 'top_left'
show(row(e,f))
sp500 = pd.DataFrame()
sp500 = yf.download('^GSPC', start='2007-01-01')['Adj Close']
sp500 = sp500.asfreq(freq='M', method='ffill')
margin_debt = final_df['NYSE Investor Margin Debt Relative To GDP'].dropna()
g = figure(title='Цепной индекс NYSE Investor Margin Debt Relative To GDP + S&P500', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
g.line(sp500.index, sp500/sp500[0], color='green', line_width=1, legend='S&P500')
#g.line(margin_debt.index, margin_debt/margin_debt[0], color='blue', line_width=1, legend='NYSE Investor Margin Debt Relative To GDP')
y_column2_range = 'NYSE Investor Margin Debt Relative To GDP'+ "_range"
g.extra_y_ranges = {y_column2_range: Range1d(start=margin_debt.min() * (1 - 0.5), end=margin_debt.max() * (1 + 0.1))}
g.add_layout(LinearAxis(y_range_name=y_column2_range), "right")
g.line(margin_debt.index, margin_debt/margin_debt[0], legend='NYSE Investor Margin Debt Relative To GDP', line_width=1, y_range_name=y_column2_range,  color="blue")
g.legend.location = 'top_left'
tobin_q = final_df['Tobin Q'].dropna()
h = figure(title='Tobin Q', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
h.line(tobin_q.index, tobin_q, color='green', line_width=1, legend='Tobin Q')
h.legend.location = 'top_left'
show(row(g,h))
sp_500_index = final_df['SP 500 Index'].dropna()
sp_500_index_gf = final_df['SP 500 GF Value'].dropna()
i = figure(title='SP 500 Index + GF', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
i.line(sp_500_index.index, sp_500_index, color='green', line_width=1, legend='SP 500 Index')
i.line(sp_500_index_gf.index, sp_500_index_gf, color='blue', line_width=1, legend='SP 500 GF Value')
i.legend.location = 'top_left'
insiders = final_df['Insider Buy/Sell Ratio - Overall Market'].dropna()
jj = figure(title='Insider Buy/Sell Ratio - Overall Market', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
jj.line(insiders.index, insiders, color='green', line_width=1, legend='Insider Buy/Sell Ratio - Overall Market')
jj.legend.location = 'top_left'
show(row(i,jj))
vix = final_df['CBOE Volatility Index (the VIX)'].dropna()
j = figure(title='CBOE Volatility Index (the VIX)', plot_height=400, plot_width=500, x_axis_label='Год', x_axis_type='datetime')
j.line(vix.index, vix, color='green', line_width=1, legend='CBOE Volatility Index (the VIX)')
j.legend.location = 'top_left'
print('Current_VIX:', vix[-1], ', Max_VIX:', vix[-252:].max(), ', Min_VIX:', vix[-252:].min())
show(j)

Loading BokehJS ...

Downloading: 100%|██████████| 524/524 [00:21<00:00, 24.79rows/s]


p2gf_value                 1.21
rank_financial_strength    4.65
rank_profitability         6.97
rank_value                 3.44
dtype: float64


Downloading: 100%|██████████| 19/19 [00:01<00:00, 11.58rows/s]


[*********************100%***********************]  1 of 1 completed


Current_VIX: 22.75 , Max_VIX: 82.69 , Min_VIX: 12.1
